In [28]:
import matplotlib
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as spl
import scipy.sparse as sp 
import matplotlib.colors as colors
import time

# Parameters

## Model parameters

In [29]:
D = 1800.00 * 1e-6
k = 4.5
Rg = 62.36*1e6 # Ideal gas constant, in mm3.mmHg.K-1.mol-1 
Tb = 309.25    # Blood temperature, 36.1*C in Kelvin
c0 = 40/(Rg*Tb) # Arterial oxygen concentration, in mol.mm-3
w = 1.00*1e-3
U = 2400.00 *1e-6
f = 44/60 * 1e-4 # Blood flow in mm3.s-1
print(c0)

2.0741691785719655e-09


## Geometric parameters

In [30]:
r = 0.005 # Radius
l = 1e-1# 50*1e-3
dx = dy = 0.0005
dt = 0.1
x = np.arange(0,l,dx)
y = np.arange(0,50*r,dy) 
L,H = x.max(), y.max()
X,Y = np.meshgrid(x,y)
nx = x.size
ny = y.size
print(x.max(), y.max())

0.0995 0.2495


# Make advection in vessel

In [31]:
gamma = U/w*dx
#C = sp.diags([f/dx,-f/dx-gamma], [-1,0], shape=(nx+1, nx+1+nx*ny), format='lil')
C = sp.diags([f/dx,-f/dx], [-1,0], shape=(nx+1, nx+1), format='lil')
C[0,0] = 1
# for i in range(nx):
#     C[1+i, nx+1+i] = gamma
C = C.tocsr()

# Make diffusion reaction matrix

In [32]:
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs) - k*dx*dy#x.max()*y.max()
offsets = np.array([-nx, -1, 0, 1, nx])
print(f"{nx=} {ny=}")

nx=200 ny=500


In [33]:
M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')
for j in range(y.size):
    for i in range(x.size):
        cell = j*nx + i
        if i==0:
            M[cell,cell] += coeffs[1]
            if cell-1 >= 0:
                M[cell, cell-1] = 0.0
        if i==nx-1:
            M[cell, cell] += coeffs[3]
            if cell+1 < M.shape[1]:
                M[cell, cell+1] = 0.0
        if j==0:
            M[cell, cell] += coeffs[0] - D*gamma
        if j==ny-1:
            M[cell, cell] += coeffs[4]

M = M.tocsr()

In [34]:
# K = sp.vstack([C,M])
# b = np.zeros(K.shape[0])
# b[0] = 1
# plt.spy(K)

# Make RHS and solve iteratively

In [35]:
# c = spl.spsolve(K, b)*Rg*Tb
# cv, ct = c[:nx+1], c[nx+1:]

In [36]:
cv = np.ones(x.size+1)*c0 # Add a ghost cell for the inlet
ct = np.ones(nx*ny)*0

bv = np.zeros(x.size+1)
bt = np.zeros(nx*ny)
bv[0] = c0

epsilon = 1e-7
err = 1
count = 0
maxcount = 20
while err > epsilon and count<maxcount:
    bv[1:] = (cv[1:]-ct[:nx])*gamma *2 # *2 to account for the symmetry (tissue on the other side)
    cvnew = spl.spsolve(C, bv)

    #bt[:nx] = (ct[:nx]-cvnew[1:])*U/w*dx
    bt[:nx] = -cvnew[1:]*gamma*D
    ctnew = spl.spsolve(M, bt)

    err = np.linalg.norm(ctnew-ct)/c0 + np.linalg.norm(cvnew-cv)/c0
    count+=1
    print(f"{count=}: {err=} {abs(ct).mean()*Rg*Tb} {abs(cv).mean()*Rg*Tb}")
    ct, cv = ctnew, cvnew

cv *= (Rg*Tb)
ct *= (Rg*Tb)

count=1: err=29.6589911550264 0.0 40.0
count=2: err=40.14050708458743 0.11201085915122776 37.815287200361794
count=3: err=35.795840766434225 0.16917348180491906 46.11342745089359
count=4: err=24.889671775782006 0.08297167204764759 29.445812995670565
count=5: err=14.069351292941562 0.09195299008225322 25.155448440745747
count=6: err=6.619261452018887 0.04290340391134844 15.543420522362078
count=7: err=2.612755116559251 0.05300374882786071 14.58445410413309
count=8: err=0.8546512319151469 0.04222740143350669 12.18314428499198
count=9: err=0.21931400698438666 0.0453641897834741 12.51104438441688
count=10: err=0.03746575672216469 0.04472083026594047 12.336433799018462
count=11: err=0.01181904663349005 0.04473977782087016 12.341576248252487
count=12: err=0.008425901244946209 0.044804034410634234 12.359015767601509
count=13: err=0.003990044856738931 0.04476297599762111 12.347872334829681
count=14: err=0.001458729617930116 0.04478030349911319 12.352575094478945
count=15: err=0.000426805074586

# Plot

In [37]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax = ax.ravel()
## Linear plot
#c = ax[0].pcolormesh(X,Y,ct.reshape(X.shape), cmap='RdBu')#, vmin=u.min(), vmax=u.max())
c = ax[0].contourf(X,Y, ct.reshape(X.shape), cmap='RdBu', levels=50,vmin=0, vmax=2)
## Log plot
#c = ax[0].pcolormesh(X,Y, ct.reshape(X.shape), cmap='RdBu', 
#                   norm=colors.LogNorm(vmin=ct.min(), vmax=ct.max()))
fig.colorbar(c, ax=ax[0], extend='max')

ax[1].plot(np.arange(cv.size)*dx-dx, cv, label='Vascular nodes O2 concentration')
plt.legend()
plt.suptitle(f"Convergence error: {err}")
plt.show()

In [38]:
# plt.figure()
# bv = np.zeros(cv.size)
# bv[0] = c0
# cv = spl.spsolve(C,bv)*Rg*Tb
# plt.plot(cv)